# Living Earth - L3+ Layers [GEE implementation]

---
This notebook is aiming to help automatizing the production of Environmental Descriptors (ED) required to produce consistent Level 3+ LCCS maps.

The LivingEarth LCCS code provides an implementation of the Food and Agriculture Organization’s (FAO’s) Land Cover Classification System (LCCS) designed to be applied to Earth Observation (EO) data. The system takes multiple products and combines these through a series of decision trees to produce all classes described as part of LCCS v2, with some modifications where required to apply to EO data.

The Food and Agriculture Organization’s (FAO’s) Land Cover Classification System (LCCS) has two stages, a Dichotomous phase (Level 3) and modular-hierarchical phase (Level 4).

More information can be found at: https://livingearth-lccs.readthedocs.io/en/latest/index.html 

This notebook uses:
- The Google Earth Engine (https://earthengine.google.com) to process satellite data 
- The Coperniucs Land Monitoring Service (CLMS) download API (https://eea.github.io/clms-api-docs/index.html) to download additional environmental layers

Author(s): <a href='https://www.unige.ch/envirospace/people/giuliani' target='_blank'>Gregory Giuliani</a>, Carole Planque, Sébastien Chognard [University of Geneva]
<br>Version: 1.0
<br>Date: 2025-01-15
<br>Supported by: Horizon-Europe LandShift; Living-Switzerland

---
<b>LCCS Level-3 Layers</b>
1. Initialize and select your sensor (Landsat or Sentinel)
2. Vegetation layer - [vegetat_veg_cat](#vegetat_veg_cat) - [Landsat 8](#veg_l8) | [Sentinel-2](#veg_s2)
3. Water layer - [aquatic_wat_cat](#aquatic_wat_cat) - [Landsat 8](#wat_l8) | [Sentinel-2](#wat_s2)
4. Cultivated layer - [cultman_agr_cat](#cultman_agr_cat) - Landsat 8 | Sentinel-2
5. Artificial layer - [artific_urb_cat](#artific_urb_cat) - [Landsat 8](#urb_l8) | [Sentinel-2](#urb_s2)

<b>Additional Layers (L3+)</b>
1. Life Form Layer - [lifeform_veg_cat](#lifeform_veg_cat)
2. Canopy cover - [canopyco_veg_con](#canopyco_veg_con)
3. Leaf Type - [leaftype_veg_cat](#leaftype_veg_cat)

---
## LCCS Level-3 Layers

### Initialize

In [1]:
import ee
import geemap

In [2]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
#Dates
startDate = '2024-05-01'
endDate = '2024-10-31'

In [4]:
#AoI using assets
aoi = ee.FeatureCollection('projects/ee-rs2/assets/switzerland')
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi, 7)

In [4]:
#AoI using bounding boxes
aoi = ee.Geometry.Polygon(
[[[6.488023408101995,46.31431816468718],
[6.798387177633245,46.31431816468718],
[6.798387177633245,46.416664011458344],
[6.488023408101995,46.416664011458344],
[6.488023408101995,46.31431816468718]]])
Map.addLayer(aoi,{},'AOI')
Map.centerObject(aoi, 12)

#### *Landsat 8 - True color*

In [5]:
#Landsat 8
#Cloud masking Landsat
def cloudMask(image):
  cloudShadowBitmask = (1 << 3)
  cloudBitmask = (1 << 5)
  qa = image.select('QA_PIXEL')
  mask = qa.bitwiseAnd(cloudShadowBitmask).eq(0) \
                .And(qa.bitwiseAnd(cloudBitmask).eq(0))
  return image.updateMask(mask)

#Import Landsat 8 image collection
CollectionL8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2") \
              .filterBounds(aoi) \
              .filterDate(startDate, endDate) \
              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
              .map(cloudMask)

#RGB visualization parameters
visualizationL8 = {
  'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
  'min': 7000,
  'max': 14000,
}

#Visualize RGB median
MedianL8 = CollectionL8.median().clip(aoi)
Map.addLayer(MedianL8, visualizationL8, 'Landsat 8 | True Color')

#### *Sentiel-2 - True color*

In [6]:
#Sentinel-2
#Cloud masking 
def cloudMask(image):
    scl = image.select('SCL')
    mask = scl.eq(3).Or(scl.gte(7).And(scl.lte(10)))
    return image.updateMask(mask.eq(0))

#Import Sentinel-2 image collection
CollectionS2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED") \
              .filterBounds(aoi) \
              .filterDate(startDate, endDate) \
              .filter(ee.Filter.calendarRange(1, 12, 'month')) \
              .map(cloudMask)

#RGB visualization parameters
visualizationS2 = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 1800,
}

#Visualize RGB median
MedianS2 = CollectionS2.median().clip(aoi)
Map.addLayer(MedianS2, visualizationS2, 'Sentinel-2 | True Color')

---
<a name="vegetat_veg_cat"></a>
### Vegetation layer - vegetat_veg_cat
Binary column indicating the present (or otherwise) of vegetation.
<br>Users can choose between NDVI, EVI, or Fractional Vegetation Cover (FVC)
<br>Users can also choose between Landsat 8 or Sentinel-2

<a name="veg_l8"></a>
#### *Landsat 8 - NDVI, EVI & FVC*

In [6]:
#Mean NDVI
ndviMean = MedianL8.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

#NDVI color palette
ndvi_palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000',
  '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301'
]

#NDVI visualization parameters
ndviPalette = {
  'min': 0,
  'max': 1,
  'palette': ndvi_palette
}

#Visualize NDVI
Map.addLayer(ndviMean, ndviPalette, 'Landsat 8 | NDVI')

In [ ]:
#Enhanced Vegetation Index (EVI)
B5 = MedianL8.select('SR_B5')
B4 = MedianL8.select('SR_B4')
B2 = MedianL8.select('SR_B2')
#evi = 2.5 * ((B5 - B4) / (B5 + 6 * B4 - 7.5 * B2 + 1))

evi = MedianL8.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': MedianL8.select('SR_B5'),
            'RED': MedianL8.select('SR_B4'),
            'BLUE': MedianL8.select('SR_B2')
        }).rename('EVI')

#EVI color palette
evi_palette = [
  'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
]

#EVI visualization parameters
eviPalette = {
  'min': 0,
  'max': 3,
  'palette': evi_palette
}

#Visualize EVI
Map.addLayer(evi, eviPalette, 'Landsat 8 | EVI')

In [ ]:
#Fractional Vegetation Cover (FVC)
ndviStats = ndviMean.reduceRegion(**{
  'reducer': ee.Reducer.minMax(),
  'geometry': aoi,
  'scale': 30,
  'maxPixels': 1e9
})
ndviMin = ee.Number(ndviStats.get('NDVI_min'))
ndviMax = ee.Number(ndviStats.get('NDVI_max'))

#Fractional Vegetation Cover
fvc = ((ndviMean.subtract(ndviMin)).divide(ndviMax.subtract(ndviMin))) \
          .pow(ee.Number(2)) \
          .rename('FVC')

Map.addLayer(fvc, ndviPalette, 'Landsat 8 | FVC')

In [ ]:
#Cleaning
#remove water; urban; bare

In [ ]:
# Export to Drive
Export.image.toDrive(**{
  'image': ndviMean, #TBC
  'description': 'Mean NDVI for Switzerland', #TBC
  'fileNamePrefix': 'ndviMean_2021', #TBC
  'region': aoi,
  'maxPixels': 1e10
})

<a name="veg_s2"></a>
#### *Sentinel-2 - NDVI, EVI & FVC*

In [7]:
#Mean NDVI
ndviMean = MedianS2.normalizedDifference(['B8', 'B4']).rename('NDVI')

#NDVI color palette
ndvi_palette = [
  'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000',
  '529400', '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301'
]

#NDVI visualization parameters
ndviPalette = {
  'min': 0,
  'max': 1,
  'palette': ndvi_palette
}

#Visualize NDVI
Map.addLayer(ndviMean, ndviPalette, 'Sentinel-2 | NDVI')

In [9]:
#Enhanced Vegetation Index (EVI)
evi = MedianS2.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
            'NIR': MedianS2.select('B8'),
            'RED': MedianS2.select('B4'),
            'BLUE': MedianS2.select('B2')
        }).rename('EVI')

#EVI color palette
evi_palette = [
  'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
]

#EVI visualization parameters
eviPalette = {
  'min': 0,
  'max': 3,
  'palette': evi_palette
}

#Visualize EVI
Map.addLayer(evi, eviPalette, 'Sentinel-2 | EVI')

In [8]:
#Fractional Vegetation Cover (FVC)
ndviStats = ndviMean.reduceRegion(**{
  'reducer': ee.Reducer.minMax(),
  'geometry': aoi,
  'scale': 10,
  'maxPixels': 1e9
})
ndviMin = ee.Number(ndviStats.get('NDVI_min'))
ndviMax = ee.Number(ndviStats.get('NDVI_max'))

#Fractional Vegetation Cover
fvc = ((ndviMean.subtract(ndviMin)).divide(ndviMax.subtract(ndviMin))) \
          .pow(ee.Number(2)) \
          .rename('FVC')

Map.addLayer(fvc, ndviPalette, 'Sentinel-2 | FVC')

In [ ]:
#Cleaning
#remove water; urban; bare

In [13]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndviMean, #TBC
  description='Vegetation Layer', #TBC
  fileNamePrefix='vegetat_veg_cat_2024', #TBC
  folder='evian',  
  region=aoi,
  scale=10,
  crs='EPSG:2154',
  maxPixels=1e10
)

---
<a name="aquatic_wat_cat"></a>
### Water layer - aquatic_wat_cat
Binary column indicating the present (or otherwise) of an aquatic land cover (e.g., water).
<br>Users can choose between NDWI, SAR or Copernicus (Water and Wetness status)
<br>Users can also choose between Landsat 8 or Sentinel-2

<a name="wat_l8"></a>
#### *Landsat 8 - NDWI*

In [6]:
#Mean NDWI
ndwiMean = MedianL8.normalizedDifference(['SR_B3', 'SR_B5']).rename('NDWI')

#NDWI visualization parameters
ndwiPalette = {
  'min': -0.05,
  'max': 0.05,
  'palette': ['#FFFFFF','#0000FF']
}

#Visualize NDWI
Map.addLayer(ndwiMean, ndwiPalette, 'Landsat 8 | NDWI')

In [ ]:
#Cleaning
#remove potential artifcats on land

In [ ]:
# Export to Drive
Export.image.toDrive(**{
  'image': ndwiMean, #TBC
  'description': 'Mean NDWI for Switzerland', #TBC
  'fileNamePrefix': 'ndwiMean_2021', #TBC
  'region': aoi,
  'maxPixels': 1e10
})

<a name="wat_s2"></a>
#### *Sentinel-2 - NDWI*

In [11]:
#Mean NDWI
ndwiMean = MedianS2.normalizedDifference(['B3', 'B8']).rename('NDWI')

#NDWI visualization parameters
ndwiPalette = {
  'min': -0.05,
  'max': 0.05,
  'palette': ['#FFFFFF','#0000FF']
}

#Visualize NDWI
Map.addLayer(ndwiMean, ndwiPalette, 'Sentinel-2 | NDWI')

In [ ]:
#Cleaning
#remove potential artifcats on land

In [12]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndwiMean, #TBC
  description='Water Layer', #TBC
  fileNamePrefix='aquatic_wat_cat_2024', #TBC
  folder='evian',  
  region=aoi,
  scale=10,
  crs='EPSG:2154',
  maxPixels=1e10
)

#### *SAR detection*
<b><span style="color:red">To be implemented</span></b>

#### *Copernicus - Water and Wetness*

https://land.copernicus.eu/api/en/products/high-resolution-layer-water-and-wetness
<br>DatasetID - UID: 0acef4675a4c4a548896cca00281f434
<br>DatasetDownloadInformationID: d170eb21-c4b4-47f5-88c7-958359704542 [2018]

In [1]:
import json
import jwt
import time
import requests

base_url = 'https://land.copernicus.eu'

In [2]:
# Load saved key from filesystem
service_key = json.load(open('livingearth_gee.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

#Authentication
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

In [ ]:
#SEARCH PRODUCTS TO DOWNLOAD
#metadata_fields=dataset_download_information > info on extracted files, e.g. based on bbox or NUTS
#metadata_fields=downloadable_files > info on prepackaged files e.g., country

url_downloadable_prepackaged = f"{base_url}/api/@search?portal_type=DataSet&b_size=100&metadata_fields=UID&metadata_fields=dataset_download_information"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_downloadable_prepackaged = requests.get(url_downloadable_prepackaged, headers=headers)

json_downloadable_prepackaged = response_downloadable_prepackaged.json()

#print(response_downloadable_prepackaged.text)

'''
for item in json_downloadable_prepackaged.get('items', []):
    print('Product information:')
    print('\n')
    print(3*' '+'Product title: "'+ item['title'].replace(':','')+'"')
    print(3*' '+'UID: "'+item['UID'].replace(' ','')+'"')
    print(3*' '+'Product link: '+ item['@id']+'')
    print('\n')
    
    for downloadable_file in item.get('downloadable_files', {}).get('items', []):
        print('Download option for "'+item['title'].replace(':','')+'":')
        display(downloadable_file)
        print(2*'\n')

    print('\n')
    print(100*'_')
'''

In [7]:
#WRITE INFORMATION ON AVAILABLE PACKAGES
import json
with open('data.json', 'w') as f:
    json.dump(json_downloadable_prepackaged, f)

In [ ]:
#Download request for a pre-packaged file
#FileID should be changed to download a specific country
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': '0acef4675a4c4a548896cca00281f434', 'FileID': 'fb1ca4a2-bcd1-4ddb-b0ba-fadc222d685c'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Download request for a extraction by NUTS
#NUTS should be changed to download a specific area
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': '0acef4675a4c4a548896cca00281f434', 'DatasetDownloadInformationID': 'd170eb21-c4b4-47f5-88c7-958359704542', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITF5'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Print Status of the download task
r = f"{base_url}/api/@datarequest_search?status=Finished_ok" #In_progress or Finished_ok or Queued
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_r = requests.get(r, headers=headers)
print(response_r.text)

In [ ]:
#Download
import json

data = json.loads(response_r.text)

url = data['71156916430']['DownloadURL'] #[orderID][DownloadURL]
r = requests.get(url)

if url.find('/'):
  download_file = url.rsplit('/', 1)[1]

open(download_file, 'wb').write(r.content)

---
<a name="cultman_agr_cat"></a>
### Cultivated layer - cultman_agr_cat
Binary column indicating the present (or otherwise) of cultivation.	
<br>Users can choose between XXX
<br>Users can also choose between Landsat 8 or Sentinel-2

<b><span style="color:red">To be implemented</span></b>
<br>EUCropMap 2018 & 2022 could be used: https://data.jrc.ec.europa.eu/collection/id-00346

In [13]:
crops2018 = ee.ImageCollection('JRC/D5/EUCROPMAP/V1')\
.filterDate('2018-01-01', '2019-01-01').first()

Map.addLayer(crops2018.clip(aoi), {}, 'EUCROPMAP 2018');

crops2022 = ee.ImageCollection('JRC/D5/EUCROPMAP/V1')\
.filterDate('2022-01-01', '2023-01-01').first()

Map.addLayer(crops2022.clip(aoi), {}, 'EUCROPMAP 2022');

In [11]:
# Create image collection of S-2 imagery for the perdiod 2018-2019
S2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
.filterDate(startDate, endDate) \
.filterBounds(aoi)

# Function to mask cloud from built-in quality band
# information on cloud

def func_lam(image):
    QA60 = image.select(['QA60'])
    return image.updateMask(QA60.lt(1))

maskcloud1 = func_lam

# Function to calculate and add an NDVI band

def func_uoh(image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']))

addNDVI = func_uoh

# Add NDVI band to image collection
S2 = S2.map(addNDVI)
# Extract NDVI band and create NDVI median composite image
NDVI = S2.select(['nd'])
# NDVI = ee.FeatureCollection(NDVI.map(mapfunc))
reducer1 = ee.Reducer.mean()

reducers = reducer1.combine(**{"reducer2": ee.Reducer.median(), "sharedInputs": True}) \
.combine(**{"reducer2": ee.Reducer.max(), "sharedInputs": True}) \
.combine(**{"reducer2": ee.Reducer.min(), "sharedInputs": True}) \
.combine(**{"reducer2": ee.Reducer.stdDev(), "sharedInputs": True})

results = NDVI.reduce(reducers)

print(results.getInfo())
Map.addLayer(results, {"bands": 'nd_stdDev', "min": 0, "max": 0.3})


{'type': 'Image', 'bands': [{'id': 'nd_mean', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'nd_median', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'nd_max', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'nd_min', 'data_type': {'type': 'PixelType', 'precision': 'float', 'min': -1, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'nd_stdDev', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


---
<a name="artific_urb_cat"></a>
### Artificial layer - artific_urb_cat
Binary column indicating the present (or otherwise) of Urban areas.	
<br>Users can choose between NDBI or Copernicus (Imperviousness Density)
<br>Users can also choose between Landsat 8 or Sentinel-2

<a name="urb_l8"></a>
#### *Landsat 8 - NDBI*

In [11]:
#Mean NDBI
ndbiMean = MedianL8.normalizedDifference(['SR_B6', 'SR_B5']).rename('NDBI')

#NDBI visualization parameters
ndbiPalette = {
  'min': 0,
  'max': 0.1,
  'palette': ['#FFFFFF','#FF0000']
}

#Visualize NDBI
Map.addLayer(ndbiMean, ndbiPalette, 'Landsat 8 | NDBI')

In [ ]:
#Cleaning
#remove potential artifiacts (remove lakes/water)

In [ ]:
# Export to Drive
Export.image.toDrive(**{
  'image': ndbiMean, #TBC
  'description': 'Mean NDBI for Switzerland', #TBC
  'fileNamePrefix': 'ndbiMean_2021', #TBC
  'region': aoi,
  'maxPixels': 1e10
})

<a name="urb_s2"></a>
#### *Sentinel-2 - NDBI*

In [14]:
#Mean NDBI
ndbiMean = MedianS2.normalizedDifference(['B12', 'B8']).rename('NDBI')

#NDBI visualization parameters
ndbiPalette = {
  'min': 0,
  'max': 0.1,
  'palette': ['#FFFFFF','#FF0000']
}

#Visualize NDBI
Map.addLayer(ndbiMean, ndbiPalette, 'Sentinel-2 | NDBI')

In [ ]:
#Cleaning
#remove potential artifiacts (remove lakes/water)

In [15]:
# Export to Drive
geemap.ee_export_image_to_drive(
  image=ndbiMean, #TBC
  description='Artificial Layer', #TBC
  fileNamePrefix='artific_urb_cat_2024', #TBC
  folder='evian',  
  region=aoi,
  scale=10,
  crs='EPSG:2154',
  maxPixels=1e10
)

#### *Copernicus Imperviousness Density*

https://land.copernicus.eu/api/en/products/high-resolution-layer-imperviousness
<br>DatasetID - UID: 4354f5ba912d4071b3b5f9300104765b
<br>DatasetDownloadInformationID: f9a55a6d-c98a-4098-bf94-f2ae7cdb0207 [2018]

In [ ]:
import json
import jwt
import time
import requests

base_url = 'https://land.copernicus.eu'

In [ ]:
# Load saved key from filesystem
service_key = json.load(open('livingearth_gee.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

#Authentication
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

In [ ]:
#Download request for a pre-packaged file
#FileID should be changed to download a specific country
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': '4354f5ba912d4071b3b5f9300104765b', 'FileID': 'fface0a2-4e04-45ef-aa54-85af8be2a6e6'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Download request for a extraction by NUTS
#NUTS should be changed to download a specific area
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': '4354f5ba912d4071b3b5f9300104765b', 'DatasetDownloadInformationID': '9a55a6d-c98a-4098-bf94-f2ae7cdb0207', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITF5'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Print Status of the download task
r = f"{base_url}/api/@datarequest_search?status=Finished_ok" #In_progress or Finished_ok or Queued
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_r = requests.get(r, headers=headers)
print(response_r.text)

In [ ]:
#Download
#orderID should be changed 
import json

data = json.loads(response_r.text)

url = data['71156916430']['DownloadURL'] #[orderID][DownloadURL]
r = requests.get(url)

if url.find('/'):
  download_file = url.rsplit('/', 1)[1]

open(download_file, 'wb').write(r.content)

---
## Additional Layers (L3+)

<a name="lifeform_veg_cat"></a>
### Life Form Layer - lifeform_veg_cat
woody (trees, shrubs), herbeceous (graminoids, forbs) & cryptograms (lichens, mosses)

Woody: https://land.copernicus.eu/api/en/products/high-resolution-layer-forest-type
<br>DatasetID - UID: 38eabc726bcc4c14a00752220df5f101
<br>DatasetDownloadInformationID: 09cbff91-1716-4dad-aecf-4f511d5a17d8 [2018]

Herbeceous: https://land.copernicus.eu/api/en/products/high-resolution-layer-grassland
<br>DatasetID - UID: bf4f7689ca1b4f3fa4a5d376e104fb49
<br>DatasetDownloadInformationID: d0dc4585-3016-4d6f-83f4-8f13cf702217 [2018]

<i>Woody</i>

In [ ]:
import json
import jwt
import time
import requests

base_url = 'https://land.copernicus.eu'

In [ ]:
# Load saved key from filesystem
service_key = json.load(open('livingearth_gee.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

#Authentication
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

In [ ]:
#Download request for a pre-packaged file
#FileID should be changed to download a specific country
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': '38eabc726bcc4c14a00752220df5f101', 'FileID': '09cbff91-1716-4dad-aecf-4f511d5a17d8'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Download request for a extraction by NUTS
#NUTS should be changed to download a specific area
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': '38eabc726bcc4c14a00752220df5f101', 'DatasetDownloadInformationID': '09cbff91-1716-4dad-aecf-4f511d5a17d8', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITF5'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Print Status of the download task
r = f"{base_url}/api/@datarequest_search?status=Finished_ok" #In_progress or Finished_ok or Queued
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_r = requests.get(r, headers=headers)
print(response_r.text)

In [ ]:
#Download
#orderID should be changed 
import json

data = json.loads(response_r.text)

url = data['71156916430']['DownloadURL'] #[orderID][DownloadURL]
r = requests.get(url)

if url.find('/'):
  download_file = url.rsplit('/', 1)[1]

open(download_file, 'wb').write(r.content)

<i>Grassland</i>

In [ ]:
import json
import jwt
import time
import requests

base_url = 'https://land.copernicus.eu'

In [ ]:
# Load saved key from filesystem
service_key = json.load(open('livingearth_gee.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

#Authentication
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

In [ ]:
#Download request for a pre-packaged file
#FileID should be changed to download a specific country
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': 'bf4f7689ca1b4f3fa4a5d376e104fb49', 'FileID': 'd0dc4585-3016-4d6f-83f4-8f13cf702217'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Download request for a extraction by NUTS
#NUTS should be changed to download a specific area
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': 'bf4f7689ca1b4f3fa4a5d376e104fb49', 'DatasetDownloadInformationID': 'd0dc4585-3016-4d6f-83f4-8f13cf702217', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITF5'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Print Status of the download task
r = f"{base_url}/api/@datarequest_search?status=Finished_ok" #In_progress or Finished_ok or Queued
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_r = requests.get(r, headers=headers)
print(response_r.text)

In [ ]:
#Download
#orderID should be changed 
import json

data = json.loads(response_r.text)

url = data['71156916430']['DownloadURL'] #[orderID][DownloadURL]
r = requests.get(url)

if url.find('/'):
  download_file = url.rsplit('/', 1)[1]

open(download_file, 'wb').write(r.content)

<a name="canopyco_veg_con"></a>
### Canopy cover - canopyco_veg_con
Canopy cover expressed in %

https://land.copernicus.eu/api/en/products/high-resolution-layer-tree-cover-density
<br>DatasetID - UID: b2227bbb31e140bdabd2fcf86a5dc3d2
<br>DatasetDownloadInformationID: f2140094-b96d-46ed-92d0-e6f2344ce3af [2018]

In [ ]:
import json
import jwt
import time
import requests

base_url = 'https://land.copernicus.eu'

In [ ]:
# Load saved key from filesystem
service_key = json.load(open('livingearth_gee.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

#Authentication
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

In [ ]:
#Download request for a pre-packaged file
#FileID should be changed to download a specific country
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': 'b2227bbb31e140bdabd2fcf86a5dc3d2', 'FileID': 'f2140094-b96d-46ed-92d0-e6f2344ce3af'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Download request for a extraction by NUTS
#NUTS should be changed to download a specific area
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': 'b2227bbb31e140bdabd2fcf86a5dc3d2', 'DatasetDownloadInformationID': 'f2140094-b96d-46ed-92d0-e6f2344ce3af', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITF5'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Print Status of the download task
r = f"{base_url}/api/@datarequest_search?status=Finished_ok" #In_progress or Finished_ok or Queued
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_r = requests.get(r, headers=headers)
print(response_r.text)

In [ ]:
#Download
#orderID should be changed 
import json

data = json.loads(response_r.text)

url = data['71156916430']['DownloadURL'] #[orderID][DownloadURL]
r = requests.get(url)

if url.find('/'):
  download_file = url.rsplit('/', 1)[1]

open(download_file, 'wb').write(r.content)

<a name="leaftype_veg_cat"></a>
### Leaf Type - leaftype_veg_cat
broad-leaved, needle-leaved or aphyllous

https://land.copernicus.eu/api/en/products/high-resolution-layer-dominant-leaf-type
<br>DatasetID - UID: b857f275e17a41a5a202348fde73c0b2
<br>DatasetDownloadInformationID: 654bcf8a-047d-46d4-81be-ba672eca1dd6 [2018]

In [ ]:
import json
import jwt
import time
import requests

base_url = 'https://land.copernicus.eu'

In [ ]:
# Load saved key from filesystem
service_key = json.load(open('livingearth_gee.json', 'rb'))

private_key = service_key['private_key'].encode('utf-8')

claim_set = {
    "iss": service_key['client_id'],
    "sub": service_key['user_id'],
    "aud": service_key['token_uri'],
    "iat": int(time.time()),
    "exp": int(time.time() + (60 * 60)),
}
grant = jwt.encode(claim_set, private_key, algorithm='RS256')

#Authentication
result = requests.post(
        service_key["token_uri"],
        headers={
            "Accept": "application/json",
            "Content-Type": "application/x-www-form-urlencoded",
        },
        data={
            "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
            "assertion": grant,
        },
)

access_token_info_json = result.json()
access_token = access_token_info_json.get('access_token')
print(access_token)

In [ ]:
#Download request for a pre-packaged file
#FileID should be changed to download a specific country
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': 'b857f275e17a41a5a202348fde73c0b2', 'FileID': '654bcf8a-047d-46d4-81be-ba672eca1dd6'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Download request for a extraction by NUTS
#NUTS should be changed to download a specific area
url_download_item = f"{base_url}/api/@datarequest_post"
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
data = {'Datasets': [{'DatasetID': 'b857f275e17a41a5a202348fde73c0b2', 'DatasetDownloadInformationID': '654bcf8a-047d-46d4-81be-ba672eca1dd6', 'OutputFormat': 'Geotiff', 'OutputGCS': 'EPSG:4326', 'NUTS': 'ITF5'}]}
response_download_item = requests.post(url_download_item, headers=headers, json=data)
print('Response:',response_download_item)
print('Message with TaskID:\n'+ response_download_item.text)

In [ ]:
#Print Status of the download task
r = f"{base_url}/api/@datarequest_search?status=Finished_ok" #In_progress or Finished_ok or Queued
headers = {'Accept': 'application/json', 'Authorization': f'Bearer {access_token}'}
response_r = requests.get(r, headers=headers)
print(response_r.text)

In [ ]:
#Download
#orderID should be changed 
import json

data = json.loads(response_r.text)

url = data['71156916430']['DownloadURL'] #[orderID][DownloadURL]
r = requests.get(url)

if url.find('/'):
  download_file = url.rsplit('/', 1)[1]

open(download_file, 'wb').write(r.content)